# Dota last picker

Dota is...

## What were we trying to solve?

Dota is...

### The actual functions

Get all the possible last picks

### Why did we solve it this way?

Dota is...

### Where did we get this data?

Dota is...

In [224]:
import numpy as np
import pandas as pd
from sklearn.naive_bayes import GaussianNB

In [225]:
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')
df_hero = pd.read_json('data/heros.json', orient='records')

In [226]:
df_hero = df_hero['heroes']

In [227]:
df_hero

0      {'name': 'antimage', 'id': 1, 'localized_name'...
1      {'name': 'axe', 'id': 2, 'localized_name': 'Axe'}
2      {'name': 'bane', 'id': 3, 'localized_name': 'B...
3      {'name': 'bloodseeker', 'id': 4, 'localized_na...
4      {'name': 'crystal_maiden', 'id': 5, 'localized...
                             ...                        
107    {'name': 'phoenix', 'id': 110, 'localized_name...
108    {'name': 'techies', 'id': 105, 'localized_name...
109    {'name': 'oracle', 'id': 111, 'localized_name'...
110    {'name': 'winter_wyvern', 'id': 112, 'localize...
111    {'name': 'arc_warden', 'id': 113, 'localized_n...
Name: heroes, Length: 112, dtype: object

In [228]:
df_test

,cluster_id,game_mode,game_type,hero_1,hero_2,hero_3,hero_4,hero_5,hero_6,hero_7,...,hero_105,hero_106,hero_107,hero_108,hero_109,hero_110,hero_111,hero_112,hero_113,outcome
0,223,8,2,0,-1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,team_2
1,227,8,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,team_1
2,136,2,2,1,0,0,0,-1,0,0,...,0,0,0,0,0,0,0,0,0,team_2
3,227,2,2,-1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,team_1
4,184,2,3,0,0,0,-1,0,0,0,...,0,0,0,0,0,0,0,0,0,team_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10289,121,2,2,0,0,0,0,0,0,1,...,-1,0,0,0,0,0,0,0,0,team_1
10290,154,9,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,team_1
10291,122,9,2,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,team_1
10292,152,2,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,team_1


In [229]:
df_train

,cluster_id,game_mode,game_type,hero_1,hero_2,hero_3,hero_4,hero_5,hero_6,hero_7,...,hero_105,hero_106,hero_107,hero_108,hero_109,hero_110,hero_111,hero_112,hero_113,outcome
0,223,2,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,team_2
1,152,2,2,0,0,0,1,0,-1,0,...,0,0,0,0,0,0,0,0,0,team_1
2,131,2,2,0,0,0,1,0,-1,0,...,0,0,0,0,0,0,0,0,0,team_1
3,154,2,2,0,0,0,0,0,0,-1,...,0,0,0,0,0,0,0,0,0,team_1
4,171,2,3,0,0,0,0,0,-1,0,...,0,0,0,0,0,0,0,0,0,team_2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92645,154,2,3,1,0,0,-1,0,0,0,...,0,0,0,0,0,0,0,0,0,team_2
92646,154,2,2,0,0,0,0,-1,0,0,...,0,0,0,0,0,0,0,0,0,team_1
92647,111,2,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,team_1
92648,185,2,2,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,team_2


In [230]:
def dataFilter(data):
    
    data = data.drop(["cluster_id", "game_mode", "game_type"], axis=1)
    features = data.drop("outcome", axis=1)
    classes = data.outcome
    normalize = []
    for i in classes:
        if(i == "team_2"):
            normalize.append(1)
        else:
            normalize.append(-1)
    return np.array(features), normalize

In [231]:
# Data preparation

df_train_features, train_classes = dataFilter(df_train)
df_test_features, test_classes = dataFilter(df_test)

In [232]:
np.array(df_train_features)[1]

array([ 0,  0,  0,  1,  0, -1,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,
        0,  0,  0, -1,  0,  0,  0,  0,  1,  1,  0,  0,  0,  0,  0,  0,  0,
       -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0, -1,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

In [233]:
train_classes[1]

-1

In [234]:
def buildClassifier(train_data):
    features, classes = dataFilter(train_data)
    # create naive bayes classifier
    gaunb = GaussianNB()
    gaunb = gaunb.fit(features, classes)
    return gaunb

In [235]:
predictor = buildClassifier(df_train)

In [236]:
#testing predictor functionality
predictor.predict(np.array(df_train_features))

array([-1, -1, -1, ..., -1,  1, -1])

In [237]:
#analysis of predictor with its own train data

score_train = predictor.score(df_train_features, train_classes)
print(score_train)

0.5729519697787372


In [238]:
#analysis of predictor with its test data

score_test = predictor.score(df_test_features, test_classes)
print(score_test)

0.5640178744899942


#### Accuracy score on this dataset

So...

### The actual functions

Assume that a player used our service on Lastpick.xyz which is our website.
We require the user to put in the HERO NAMES in this order
Example : ( Anti-mage, Axe, Bane, Slark, Crystal Maiden, Orge Magi, Queen Of Pain, Beastmaster, Monkey king)

The first 5 heroes are what the enemies picked, and the 4 last heroes are current picked heroes on your team.

getMyLastpick(pick) will return all the possible picks that the user can consider picking according to what have been picked.

heroNameTranslate(hero) will return an id given the name (assume that the name is correct).


In [269]:
test_case1 = ["Axe","Anti-Mage","Bane","Crystal Maiden","Queen of Pain"
              ,"Lone Druid", "Tinker","Abyssal Underlord","Phoenix"]
test_case2 = ["Axe","Juggernaut","Bane","Crystal Maiden","Queen of Pain"
              ,"Spectre", "Jakiro","Abyssal Underlord","Phoenix"]

In [270]:
df_hero_idname = pd.json_normalize(df_hero)[["localized_name","id"]]

In [281]:
def getHeroIds(draft):
    answer = []
    for i in draft:
        answer.append(int(df_hero_idname.loc[df_hero_idname['localized_name'] == i]['id']))
    return answer

In [282]:
getHeroNames(test_case1)

[2, 1, 3, 5, 39, 80, 34, 108, 110]

In [170]:
def getMyLastPick(picks, clf):
    """
    return last pick given (1,2,3,4,5,6,7,8,9) [1,2,3,4,5] [6,7,8,9,?] <-- your team
    for i in range(1,114) try everything but 
    1. cannot pick currently selected heroes
    """
    possiblePicks = []
    for i in range(1,114):
        trial = picks.copy()
        if(i not in picks):
            trial.append(i)
            result = clf.predict(trial)
            if(result == np.array(1)):
                possiblePicks.append(trial)
    return possiblePicks